# Limpieza de base de datos tipo texto estado: caótico 

Ejemplo de **limpieza avanzada de datos desordenados**
provenientes de un archivo de Excel. 
El archivo contiene información mezclada en una sola columna sobre:
- Nombres y apellidos
- Roles
- Ciudades
- Correos electrónicos
- Texto sucio y símbolos extra

## Objetivo

Transformar el archivo “corrupto” en una tabla limpia y estructurada, en tipo de archivo excel.

## Acciones 

# Carga de archivo y exploración inicial

- carga de librerías
- visualización 
- estructura y tipos de dato

In [1]:
# importar librerías
import pandas as pd
import numpy as np
import re

In [2]:
# carga de datos 

file_path = r"C:\Users\andre\OneDrive\Data_Cleaning_Portfolio\Datos_tipo_texto\datos_originales\corrupted_csv_example.xlsx"
df_raw = pd.read_excel(file_path)
df = df_raw.copy()
df_raw.head()

,Corrupted_Row
0,Paola López::Support Agent::Manta::paola0@g ma...
1,Brian Ramirez::Developer::Manta::brian1@g mail...
2,Eduardo Bravo;Product Owner;Daule;eduardo2@g m...
3,\tMarioSánchez\tQALead\tManta\tmario3@gmail.com
4,Cecilia---Torres|Developer|Tena|cecilia4@g---m...


In [3]:
# exploracion del set de datos 
print("Shape del dataframe:", df.shape)
print("Columnas:", df.columns.tolist())

# Mostrar algunas filas de ejemplo
pd.set_option("display.max_colwidth", 200)
df.sample(10, random_state=42)

Shape del dataframe: (50, 1)
Columnas: ['Corrupted_Row']


,Corrupted_Row
13,Hector Jaramillo|IT Lead|Ambato|hector13@g mail . com
39,"Javier Moreno,HR,Milagro,javier39@g mail . com"
30,Roberto Casas\tQA Lead\tManta\troberto30@g mail . com
45,Claudia Pérez::Developer::Riobamba::claudia45@g mail . com:: note:fix-this
17,Javier MorenoQA LeadMachalajavier17@g mail . com note:fix-this
48,Marcelo Paredes Sales Manta marcelo48@g mail . com
26,Melissa Viteri Marketing Tena melissa26@g mail . com
25,Juan Ruiz;IT Lead;Cuenca;juan25@g mail . com; note:fix-this
32,Claudia---Pérez|Developer|Manta|claudia32@g---mail---.---com
19,"Andrea Molina,Sales,Daule,andrea19@g mail . com, note:fix-this"


# Extracción y normalización de correo electrónico 

- normalización de variantes de gmail
- Extraer correo a nueva columna
- Eliminar texto residual
- Se crea una columna sin el email

In [4]:
#función que reemplaza variantes de gmail
def normalize_gmail(text: str) -> str:
    
    if not isinstance(text, str):
        text = str(text)
    # Patrón para g[algo]mail[algo].com con espacios, guiones o underscores
    pattern = re.compile(r'g[\s\-_]*mail[\s\-_]*[\.\-_\s]*com', re.IGNORECASE)
    return pattern.sub('gmail.com', text)

# extrae y devuelve el correo limpio 
def extract_email_and_rest(text: str):
    
    if not isinstance(text, str):
        text = str(text)
    t = normalize_gmail(text)

    # Buscar 'usuario@gmail.com'
    m = re.search(r'([\w\.]+@gmail\.com)', t)
    email = m.group(1) if m else None

    if email:
        rest = t.replace(email, " ")
    else:
        rest = t

    # Quitar la marca de texto basura
    rest = re.sub(r'\bnote:fix-this\b', " ", rest, flags=re.IGNORECASE)

    # Limpiar espacios extra
    rest = re.sub(r'\s+', " ", rest).strip()

    return email, rest


# Aplicar la función a cada fila
df["email"], df["text_without_email"] = zip(*df["Corrupted_Row"].map(extract_email_and_rest))


df[["Corrupted_Row", "email", "text_without_email"]].head(10)


,Corrupted_Row,email,text_without_email
0,Paola López::Support Agent::Manta::paola0@g mail . com,paola0@gmail.com,Paola López::Support Agent::Manta::
1,Brian Ramirez::Developer::Manta::brian1@g mail . com,brian1@gmail.com,Brian Ramirez::Developer::Manta::
2,Eduardo Bravo;Product Owner;Daule;eduardo2@g mail . com,eduardo2@gmail.com,Eduardo Bravo;Product Owner;Daule;
3,\tMarioSánchez\tQALead\tManta\tmario3@gmail.com,mario3@gmail.com,MarioSánchez QALead Manta
4,Cecilia---Torres|Developer|Tena|cecilia4@g---mail---.---com| note:fix-this,cecilia4@gmail.com,Cecilia---Torres|Developer|Tena| |
5,Juan Carlos MenaSalesDaulejuan5@g mail . com note:fix-this,MenaSalesDaulejuan5@gmail.com,Juan Carlos
6,"Patricia---Jaramillo,HR,Puyo,patricia6@g---mail---.---com",patricia6@gmail.com,"Patricia---Jaramillo,HR,Puyo,"
7,Rafael Paredes HR Cuenca rafael7@g mail . com,rafael7@gmail.com,Rafael Paredes HR Cuenca
8,Hector Jaramillo Manager Machala hector8@g mail . com,hector8@gmail.com,Hector Jaramillo Manager Machala
9,Veronica Lasso;Developer;Daule;veronica9@g mail . com,veronica9@gmail.com,Veronica Lasso;Developer;Daule;


## Limpieza básica de texto

- Reemplazar tabuladores, comas, barras y guiones
- Eliminar caracteres repetidos
- Reemplazar guiones bajos por espacios
- Normalizar espacios múltiples
- Crear columna nueva de texto limpio

In [5]:
# Separación de texto 
def split_name_role_city(text: str):
    
    if not isinstance(text, str):
        text = "" if pd.isna(text) else str(text)

    t = text.strip()
    if not t:
        return pd.Series([None, None, None])

    
    # 1. Normalización de separadores
    t = t.replace("\t", "|")   # tab
    t = t.replace("---", " ")  # guiones largos pegados
    t = t.replace("_", " ")    # underscores
    
    # Unificar varios separadores a '|'
    for sep in ["::", ";", "|", ","]:
        t = t.replace(sep, "|")

    # Limpiar espacios múltiples
    t = re.sub(r"\s+", " ", t).strip()

    
    # 2. División por separador '|'
    parts = [p.strip() for p in t.split("|") if p.strip()]

    # Caso ideal: 3 partes
    if len(parts) >= 3:
        return pd.Series([parts[0], parts[1], parts[2]])

    # Si hay 1 sola parte, intentar separar por bloques de espacios
    if len(parts) == 1:
        subparts = [p.strip() for p in re.split(r"\s{2,}", parts[0]) if p.strip()]
        if len(subparts) >= 3:
            return pd.Series(subparts[:3])
        elif len(subparts) == 2:
            return pd.Series([subparts[0], subparts[1], None])
        else:
            return pd.Series([parts[0], None, None])

    # Si hay 2 partes → name y role/city
    if len(parts) == 2:
        return pd.Series([parts[0], parts[1], None])

    # Incompleto
    return pd.Series([None, None, None])


# Aplicar al dataframe
df[["name", "role", "city"]] = df["text_without_email"].apply(split_name_role_city)

# Mostrar avances
df[["Corrupted_Row", "name", "role", "city", "email"]].head(12)


,Corrupted_Row,name,role,city,email
0,Paola López::Support Agent::Manta::paola0@g mail . com,Paola López,Support Agent,Manta,paola0@gmail.com
1,Brian Ramirez::Developer::Manta::brian1@g mail . com,Brian Ramirez,Developer,Manta,brian1@gmail.com
2,Eduardo Bravo;Product Owner;Daule;eduardo2@g mail . com,Eduardo Bravo,Product Owner,Daule,eduardo2@gmail.com
3,\tMarioSánchez\tQALead\tManta\tmario3@gmail.com,MarioSánchez QALead Manta,None,None,mario3@gmail.com
4,Cecilia---Torres|Developer|Tena|cecilia4@g---mail---.---com| note:fix-this,Cecilia Torres,Developer,Tena,cecilia4@gmail.com
5,Juan Carlos MenaSalesDaulejuan5@g mail . com note:fix-this,Juan Carlos,None,None,MenaSalesDaulejuan5@gmail.com
6,"Patricia---Jaramillo,HR,Puyo,patricia6@g---mail---.---com",Patricia Jaramillo,HR,Puyo,patricia6@gmail.com
7,Rafael Paredes HR Cuenca rafael7@g mail . com,Rafael Paredes HR Cuenca,None,None,rafael7@gmail.com
8,Hector Jaramillo Manager Machala hector8@g mail . com,Hector Jaramillo Manager Machala,None,None,hector8@gmail.com
9,Veronica Lasso;Developer;Daule;veronica9@g mail . com,Veronica Lasso,Developer,Daule,veronica9@gmail.com


In [6]:
pd.set_option('display.max_rows', None)       # Muestra todas las filas
pd.set_option('display.max_columns', None)    # Muestra todas las columnas
pd.set_option('display.max_colwidth', None)   # No corta el texto dentro de las celdas

df


,Corrupted_Row,email,text_without_email,name,role,city
0,Paola López::Support Agent::Manta::paola0@g mail . com,paola0@gmail.com,Paola López::Support Agent::Manta::,Paola López,Support Agent,Manta
1,Brian Ramirez::Developer::Manta::brian1@g mail . com,brian1@gmail.com,Brian Ramirez::Developer::Manta::,Brian Ramirez,Developer,Manta
2,Eduardo Bravo;Product Owner;Daule;eduardo2@g mail . com,eduardo2@gmail.com,Eduardo Bravo;Product Owner;Daule;,Eduardo Bravo,Product Owner,Daule
3,\tMarioSánchez\tQALead\tManta\tmario3@gmail.com,mario3@gmail.com,MarioSánchez QALead Manta,MarioSánchez QALead Manta,None,None
4,Cecilia---Torres|Developer|Tena|cecilia4@g---mail---.---com| note:fix-this,cecilia4@gmail.com,Cecilia---Torres|Developer|Tena| |,Cecilia Torres,Developer,Tena
5,Juan Carlos MenaSalesDaulejuan5@g mail . com note:fix-this,MenaSalesDaulejuan5@gmail.com,Juan Carlos,Juan Carlos,None,None
6,"Patricia---Jaramillo,HR,Puyo,patricia6@g---mail---.---com",patricia6@gmail.com,"Patricia---Jaramillo,HR,Puyo,",Patricia Jaramillo,HR,Puyo
7,Rafael Paredes HR Cuenca rafael7@g mail . com,rafael7@gmail.com,Rafael Paredes HR Cuenca,Rafael Paredes HR Cuenca,None,None
8,Hector Jaramillo Manager Machala hector8@g mail . com,hector8@gmail.com,Hector Jaramillo Manager Machala,Hector Jaramillo Manager Machala,None,None
9,Veronica Lasso;Developer;Daule;veronica9@g mail . com,veronica9@gmail.com,Veronica Lasso;Developer;Daule;,Veronica Lasso,Developer,Daule


In [7]:
# texto uniforme 
def normalize_basic(text):
    
    if not isinstance(text, str):
        text = str(text)

    t = text.replace("\t", " ")
    t = t.replace("|", " ")
    t = t.replace(";", " ")
    t = t.replace(",", " ")   
    t = t.replace("::", " ")
    t = t.replace("___", " ")
    t = t.replace("---", " ")
    t = t.replace("_", " ")  

    t = re.sub(r"\s+", " ", t).strip()
    return t

df["clean_text"] = df["text_without_email"].apply(normalize_basic)


## LISTAS Y FUNCIONES DE SEPARACIÓN ROL, NOMBRE Y CIUDAD

- Lista de ciudades detectadas en el archivo
- Lista de roles y palabras clave
- Diccionario para estandarizar variaciones de roles
- Generar columnas para cada item requerido

In [8]:
# Lista de ciudades conocidas
CITIES = [
    "Ambato", "Cuenca", "Daule", "Guayaquil", "Latacunga", "Loja",
    "Manta", "Milagro", "Puyo", "Quito", "Riobamba", "Tena", "Machala"
]

# Keywords de roles
ROLE_KEYWORDS = [
    "Developer",
    "Finance",
    "Director",
    "HR",
    "IT",
    "Lead",
    "Manager",
    "Product",
    "Owner",
    "QA",
    "Sales",
    "Support",
    "Agent",
    "Marketing"
]

ROLE_NORMALIZATION = {
    "ProductOwner": "Product Owner",
    "Product_Owner": "Product Owner",
    "QALead": "QA Lead"
}
def smart_split_camel(text):
    """
    Separa palabras pegadas basándose en CamelCase:
    Ej: 'MenaSalesDaule' → 'Mena Sales Daule'
    """
    return re.sub(r'(?<=[a-záéíóúñ])(?=[A-ZÁÉÍÓÚÑ])', ' ', text)

def advanced_split(text: str):
    if not isinstance(text, str):
        return pd.Series([None, None, None])

    # separar camelCase (MarioSánchez, MenaSalesDaule, etc.)
    t = smart_split_camel(text)
    # limpiar espacios extra
    t = re.sub(r"\s+", " ", t).strip()
    words = t.split()

    name_parts = []
    role_parts = []
    city = None

    for w in words:
        # normalizar variantes raras primero
        w_norm = ROLE_NORMALIZATION.get(w, w)

        # 1) CIUDADES
        if w_norm.capitalize() in CITIES:
            city = w_norm.capitalize()
            continue

        # 2) ROLES (por palabras clave)
        if w_norm in ROLE_KEYWORDS:
            role_parts.append(w_norm)
            continue

        # 3) SI NO ES NI CIUDAD NI ROL → NOMBRE
        name_parts.append(w_norm)

    name = " ".join(name_parts) if name_parts else None
    role = " ".join(role_parts) if role_parts else None

    return pd.Series([name, role, city])


In [9]:
df[["name", "role", "city"]] = df["clean_text"].apply(advanced_split)

df[["clean_text", "name", "role", "city", "email"]].head(20)


,clean_text,name,role,city,email
0,Paola López Support Agent Manta,Paola López,Support Agent,Manta,paola0@gmail.com
1,Brian Ramirez Developer Manta,Brian Ramirez,Developer,Manta,brian1@gmail.com
2,Eduardo Bravo Product Owner Daule,Eduardo Bravo,Product Owner,Daule,eduardo2@gmail.com
3,MarioSánchez QALead Manta,Mario Sánchez QA Lead,None,Manta,mario3@gmail.com
4,Cecilia Torres Developer Tena,Cecilia Torres,Developer,Tena,cecilia4@gmail.com
5,Juan Carlos,Juan Carlos,None,None,MenaSalesDaulejuan5@gmail.com
6,Patricia Jaramillo HR Puyo,Patricia Jaramillo,HR,Puyo,patricia6@gmail.com
7,Rafael Paredes HR Cuenca,Rafael Paredes,HR,Cuenca,rafael7@gmail.com
8,Hector Jaramillo Manager Machala,Hector Jaramillo,Manager,Machala,hector8@gmail.com
9,Veronica Lasso Developer Daule,Veronica Lasso,Developer,Daule,veronica9@gmail.com


## Exploración de archivo

In [10]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 500)

df_problem = df[df[["name", "role", "city"]].isna().any(axis=1)]
df_problem


,Corrupted_Row,email,text_without_email,name,role,city,clean_text
3,\tMarioSánchez\tQALead\tManta\tmario3@gmail.com,mario3@gmail.com,MarioSánchez QALead Manta,Mario Sánchez QA Lead,None,Manta,MarioSánchez QALead Manta
5,Juan Carlos MenaSalesDaulejuan5@g mail . com note:fix-this,MenaSalesDaulejuan5@gmail.com,Juan Carlos,Juan Carlos,None,None,Juan Carlos
17,Javier MorenoQA LeadMachalajavier17@g mail . com note:fix-this,LeadMachalajavier17@gmail.com,Javier MorenoQA,Javier Moreno,QA,None,Javier MorenoQA
43,Luis GomezSupport AgentTenaluis43@g mail . com,AgentTenaluis43@gmail.com,Luis GomezSupport,Luis Gomez,Support,None,Luis GomezSupport


In [11]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 500)

df_problem = df[df[["name", "role", "city"]].isna().any(axis=1)][
    ["Corrupted_Row", "clean_text", "name", "role", "city", "email"]
]
df_problem


,Corrupted_Row,clean_text,name,role,city,email
3,\tMarioSánchez\tQALead\tManta\tmario3@gmail.com,MarioSánchez QALead Manta,Mario Sánchez QA Lead,None,Manta,mario3@gmail.com
5,Juan Carlos MenaSalesDaulejuan5@g mail . com note:fix-this,Juan Carlos,Juan Carlos,None,None,MenaSalesDaulejuan5@gmail.com
17,Javier MorenoQA LeadMachalajavier17@g mail . com note:fix-this,Javier MorenoQA,Javier Moreno,QA,None,LeadMachalajavier17@gmail.com
43,Luis GomezSupport AgentTenaluis43@g mail . com,Luis GomezSupport,Luis Gomez,Support,None,AgentTenaluis43@gmail.com


## Exportación de archivo

In [ ]:
df.to_csv("contacts_clean.csv", index=False, encoding="utf-8")


In [ ]:
df.to_excel("contacts_clean.xlsx", index=False)
